In [14]:
import requests
import pprint
from datetime import datetime, timezone, timedelta

In [15]:
today = datetime.now().strftime('%Y-%m-%d')

url = f"https://statsapi.mlb.com/api/v1/schedule?sportId=1&date={today}&hydrate=probablePitcher(note,stats,person),decisions&language=en"
response = requests.get(url)
data = response.json()

In [16]:
# Perhaps change this to https://statsapi.mlb.com/api/v1/schedule?sportId=1

games = data['dates'][0]['games']
game_ids = [game['gamePk'] for game in games]

In [17]:
time_left = (datetime.strptime(games[0]['gameDate'],"%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc) - datetime.now(timezone.utc))
year = datetime.now().year

In [18]:
streaks = {}

# Can possibly put both league IDs into one URL
# Potentiall can hydrate
for league in [103,104]:
    url = f"https://statsapi.mlb.com/api/v1/standings?leagueId={league}&season={year}&type=regularSeason"
    response = requests.get(url)
    data = response.json()   
    for division in data['records']:
        for team in division['teamRecords']:
            streaks[team['team']['id']] = team['streak']['streakCode']

In [19]:
time_left = (datetime.strptime(games[0]['gameDate'],"%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc) - datetime.now(timezone.utc))
year = datetime.now().year

if time_left.days < 0:
    print("Baseball has started!")
else:
     hours = time_left.seconds // 3600
     minutes = (time_left.seconds % 3600) // 60
     seconds = (time_left.seconds % 60)
     print(f"Baseball starts in {hours} hours, {minutes} minutes and {seconds} seconds")

Baseball has started!


In [20]:
def pitcher_stats(pitcher_id):
    url = f"https://statsapi.mlb.com/api/v1/people/{pitcher_id}/stats?stats=season&group=pitching"
    response = requests.get(url)
    data = response.json()
    stats = data['stats'][0]['splits'][0]['stat']
    stats['name'] = data['stats'][0]['splits'][0]['player']['fullName']
    try:
        stats['team'] = data['stats'][0]['splits'][0]['team']['name']
    except:
        stats['team'] = data['stats'][0]['splits'][1]['team']['name']
    return(stats)

In [21]:
def game_details(game_id):
    url = f"https://statsapi.mlb.com/api/v1.1/game/{game_id}/feed/live"
    response = requests.get(url)
    data = response.json()
    return(data)

In [22]:
game_ids = [game['gamePk'] for game in games]
for game in games:
    game_id = game['gamePk']
    teams = game['teams']
    away_team = teams['away']['team']['name']
    home_team = teams['home']['team']['name']
    away_id = teams['away']['team']['id']
    home_id = teams['home']['team']['id']
    try:
        home_pitcher_name = teams['home']['probablePitcher']['fullName']
    except:
        home_pitcher_name = "TBD"
    try:
        home_pitcher_stats = pitcher_stats(teams['home']['probablePitcher']['id'])
    except:
        home_pitcher_stats = {}
    try:
        away_pitcher_name = teams['away']['probablePitcher']['fullName']
    except:
        away_pitcher_name = "TBD"
    try:
        away_pitcher_stats = pitcher_stats(teams['away']['probablePitcher']['id'])
    except:
        away_pitcher_stats = {}
    home_record = game['teams']['home']['leagueRecord']
    away_record = game['teams']['away']['leagueRecord']
    game_time = datetime.strptime(game['gameDate'],'%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=timezone.utc).astimezone(tz=None)
    if game['status']['abstractGameState'] == 'Preview' or game['status']['detailedState'] == 'Warmup':
        print(f"{game_time.strftime('%I:%M %p')}:")
        try:
            print(f"{away_team} ({away_record['wins']}-{away_record['losses']}, {streaks[away_id]}) -- {away_pitcher_name} ({away_pitcher_stats['wins']}-{away_pitcher_stats['losses']}, {away_pitcher_stats['era']} ERA)")
        except:
            print(f"{away_team} ({away_record['wins']}-{away_record['losses']}, {streaks[away_id]}) -- {away_pitcher_name}")
        print("vs.")
        try:
            print(f"{home_team} ({home_record['wins']}-{home_record['losses']}, {streaks[home_id]}) -- {home_pitcher_name} ({home_pitcher_stats['wins']}-{home_pitcher_stats['losses']}, {home_pitcher_stats['era']} ERA)")
        except:
            print(f"{home_team} ({home_record['wins']}-{home_record['losses']}, {streaks[home_id]}) -- {home_pitcher_name}")
    
    elif game['status']['abstractGameState'] == 'Live':
        url = f"https://statsapi.mlb.com/{game['link']}"
        response = requests.get(url)
        data = response.json()

        home_score = game['teams']['home']['score']
        away_score = game['teams']['away']['score']
        inning = f"{data['liveData']['linescore']['inningHalf']} {data['liveData']['linescore']['currentInningOrdinal']}"
        print(f"{inning}:")
        print(f"{away_team} {away_score}, {home_team} {home_score}")
        
        print(f"{data['liveData']['plays']['currentPlay']['matchup']['pitcher']['fullName']} pitching to {data['liveData']['plays']['currentPlay']['matchup']['batter']['fullName']}.")
        try:
            print(data['liveData']['plays']['currentPlay']['result']['description'])
            try:
                print(f"Exit Velocity: {data['liveData']['plays']['currentPlay']['playEvents'][-1]['hitData']['launchSpeed']} MPH")
            except: pass
        except:
            try:
                print(data['liveData']['plays']['allPlays'][-2]['result']['description']) 
            except:
                print("Start of game")
            try:
                print(f"Exit Velocity: {data['liveData']['plays']['allPlays'][-2]['playEvents'][-1]['hitData']['launchSpeed']} MPH")
            except: pass
                
        url = f"https://statsapi.mlb.com/api/v1/game/{game_id}/winProbability"
        response = requests.get(url)
        data = response.json()
        probability = round(data[-1]['homeTeamWinProbability'],1)
        if probability < 50:
            print(f"The {away_team} have a {100-probability}% chance of winning.")
        else:
            print(f"The {home_team} have a {probability}% chance of winning.")

    elif game['status']['abstractGameState'] == 'Final':
        home_score = game['teams']['home']['score']
        away_score = game['teams']['away']['score']
        print("Final")
        print(f"{away_team}: {away_score} | {away_record['wins']}-{away_record['losses']}, {streaks[away_id]}")
        print(f"{home_team}: {home_score} | {home_record['wins']}-{home_record['losses']}, {streaks[home_id]}")

        url = f"https://statsapi.mlb.com/api/v1/game/{game_id}/content"
        response = requests.get(url)
        data = response.json()
        try:
            print(data['editorial']['recap']['mlb']['headline'])
        except: pass
        try:
            print("Play of the game:", data['highlights']['highlights']['items'][0]['playbacks'][0]['url'])
        except: pass
            
        try:
            wp = game['decisions']['winner']['fullName']
            lp = game['decisions']['loser']['fullName']
            try:
                sv = game['decisions']['save']['fullName']
                print(f"W: {wp} || L: {lp} || SV: {sv}")
            except:
                print(f"W: {wp} || L: {lp}")
        except: pass

        url = f"https://statsapi.mlb.com/api/v1/game/{game_id}/boxscore"
        response = requests.get(url)
        data = response.json()
        for player in data['topPerformers']:
            name = player['player']['person']['fullName']
            try:
                stats = player['player']['stats']['pitching']['summary']
            except: 
                stats = player['player']['stats']['batting']['summary']    
            print(f"{name}: {stats}")            
        
    print("--------------------------")

Final
Toronto Blue Jays: 1 | 54-39, L1
Chicago White Sox: 2 | 31-62, W1
Houser battles Jays, heat in gem to end White Sox skid
Play of the game: https://mlb-cuts-diamond.mlb.com/FORGE/2025/2025-07/09/b6fdfd5a-448e1ec2-5085c734-csvm-diamondgcp-asset_1280x720_59_4000K.mp4
W: Adrian Houser || L: Eric Lauer || SV: Jordan Leasure
Adrian Houser: 7.0 IP, ER, 2 K, 2 BB
Bo Bichette: 3-3 | 2B, BB
Edgar Quero: 2-3 | 2 2B, HBP, RBI
--------------------------
Final
Los Angeles Dodgers: 2 | 56-38, L6
Milwaukee Brewers: 3 | 53-40, W4
Crew celebrates Chourio's 1st walk-off, sweep with Megill's All-Star nod
Play of the game: https://mlb-cuts-diamond.mlb.com/FORGE/2025/2025-07/09/3b8996e7-a79d95fa-fffc7717-csvm-diamondgcp-asset_1280x720_59_4000K.mp4
W: Trevor Megill || L: Kirby Yates
Jose Quintana: 6.0 IP, ER, 4 K, 4 BB
Tyler Glasnow: 5.0 IP, 0 ER, 5 K, 3 BB
Miguel Rojas: 2-3 | BB, 2 R, SB
--------------------------
Final
Philadelphia Phillies: 13 | 54-39, W1
San Francisco Giants: 0 | 51-43, L1
Play of 

KeyError: 'score'

In [ ]:
games[3]

{'gamePk': 777182,
 'gameGuid': '9fa91789-6f9d-4e61-9673-7c37421fc90f',
 'link': '/api/v1.1/game/777182/feed/live',
 'gameType': 'R',
 'season': '2025',
 'gameDate': '2025-07-09T21:10:00Z',
 'officialDate': '2025-07-09',
 'status': {'abstractGameState': 'Live',
  'codedGameState': 'I',
  'detailedState': 'In Progress',
  'statusCode': 'I',
  'startTimeTBD': False,
  'abstractGameCode': 'L'},
 'teams': {'away': {'leagueRecord': {'wins': 49, 'losses': 43, 'pct': '.533'},
   'score': 7,
   'team': {'id': 139, 'name': 'Tampa Bay Rays', 'link': '/api/v1/teams/139'},
   'probablePitcher': {'id': 641793,
    'fullName': 'Zack Littell',
    'link': '/api/v1/people/641793'},
   'splitSquad': False,
   'seriesNumber': 30},
  'home': {'leagueRecord': {'wins': 59, 'losses': 34, 'pct': '.634'},
   'score': 3,
   'team': {'id': 116, 'name': 'Detroit Tigers', 'link': '/api/v1/teams/116'},
   'probablePitcher': {'id': 681857,
    'fullName': 'Reese Olson',
    'link': '/api/v1/people/681857'},
   'spl

In [24]:
games[-1]

{'gamePk': 777181,
 'gameGuid': 'c1ae5647-41b5-4790-a7e6-16561c1b37e6',
 'link': '/api/v1.1/game/777181/feed/live',
 'gameType': 'R',
 'season': '2025',
 'gameDate': '2025-07-09T23:05:00Z',
 'officialDate': '2025-07-10',
 'rescheduleDate': '2025-07-10T21:05:00Z',
 'rescheduleGameDate': '2025-07-10',
 'status': {'abstractGameState': 'Final',
  'codedGameState': 'D',
  'detailedState': 'Postponed',
  'statusCode': 'DR',
  'startTimeTBD': False,
  'reason': 'Rain',
  'abstractGameCode': 'F'},
 'teams': {'away': {'leagueRecord': {'wins': 53, 'losses': 39, 'pct': '.576'},
   'team': {'id': 121, 'name': 'New York Mets', 'link': '/api/v1/teams/121'},
   'probablePitcher': {'id': 656849,
    'fullName': 'David Peterson',
    'link': '/api/v1/people/656849'},
   'splitSquad': False,
   'seriesNumber': 30},
  'home': {'leagueRecord': {'wins': 40, 'losses': 50, 'pct': '.444'},
   'team': {'id': 110,
    'name': 'Baltimore Orioles',
    'link': '/api/v1/teams/110'},
   'probablePitcher': {'id': 60